CUSTOMER SEGMENTATION USING RFM

Data Description

InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.

StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.

Description: Product (item) name. Nominal.

Quantity: The quantities of each product (item) per transaction. Numeric.

InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.

UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).

CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.

Country: Country name. Nominal. The name of the country where a customer resides.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import datetime as dt
pd.set_option("display.max_columns", 10)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [2]:
df_rtl = pd.read_excel("../input/uci-online-retail-ii-data-set/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_rtl.copy()

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


In [4]:
# Number of unique products
df["Description"].nunique()

4223

In [5]:
#  Number of each product:
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
Name: Description, dtype: int64

In [6]:
# Quantity of each product sold:
df.groupby("Description").agg({"Quantity": "sum"})
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039
...,...
Damaged,-7540
Printing smudges/thrown away,-9058
check,-12030


Products seen as minus are the ones who are returned. We have to eliminate the returned products by invoice.

In [7]:
# Invoice number:
df["Invoice"].nunique()

25900

DATA PREPARATION

In [8]:
# Taking only UK customers since they consitute the bigger part of the data
df["Country"].value_counts()

United Kingdom          495478
Germany                   9495
France                    8558
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [9]:
df = df[df["Country"] == "United Kingdom"]
df.shape
# we have now only the Unitem Kingdom Customers

(495478, 8)

In [10]:
# We get rid of returned invoices
df = df[~(df["Invoice"].str.contains("C", na=False))]
df.shape

(487622, 8)

In [11]:
# We get rid of null values 
df.dropna(inplace=True)
df.shape

(354345, 8)

In [12]:
# We add Total Price column to calculate the total earning for the firm
df["TotalPrice"] = df["Price"] * df["Quantity"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34


In [13]:
# Describe the data - looking for anormalities
# df.describe().T
df.describe([0.01, 0.10, 0.25, 0.50, 0.75, 0.90, 0.99]).T

,count,mean,std,min,1%,...,50%,75%,90%,99%,max
Quantity,354345.00,12.05,190.43,1.00,1.00,...,4.00,12.00,24.00,100.00,80995.00
Price,354345.00,2.96,17.86,0.00,0.21,...,1.95,3.75,5.95,12.75,8142.75
Customer ID,354345.00,15552.44,1594.55,12346.00,12748.00,...,15522.00,16931.00,17774.00,18223.00,18287.00
TotalPrice,354345.00,20.63,326.03,0.00,0.55,...,10.20,17.70,32.70,179.00,168469.60


In [14]:
# We can see that there are outliers in the price column for example since the mean 
# is around 3 and the 99th percent of the data is 12.75. 

CALCULATING RFM METRICS


In [15]:
# Recency: time passed since the "last contact" of the customer
# = Today's date - Last purchase date

In [16]:
# Lets look at maximum date on data
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:49:00')

In [17]:
# To avoid recency value to become "0" we add 1 or 2 to today's date
today_date = dt.datetime(2011, 12, 11)

In [18]:
# Creating RFM

In [19]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda date: (today_date - date.max()).days,
                                     "Invoice": lambda num: len(num), #nunique() can be used but than ranking should be defined
                                     "TotalPrice": lambda TotalPrice: TotalPrice.sum()})
rfm

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00,326,1,77183.60
12747.00,3,103,4196.01
12748.00,1,4596,33719.73
12749.00,4,199,4090.88
12820.00,4,59,942.34
...,...,...,...
18280.00,278,10,180.60
18281.00,181,7,80.82
18282.00,8,12,178.05


In [20]:
# Now, we know every customer's spending, last shopping date and how often s/he shops

In [21]:
rfm.columns = ["Recency", "Frequency", "Monetary"]

In [22]:
# We take the monetary and frequency values greater than zero
rfm = rfm[(rfm["Monetary"] > 0) & (rfm["Frequency"] > 0)]

CALCULATING RFM SCORES

In [23]:
# Recency
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])
# We divide the recency into 5 equal pieces and label them from min to max as 5 to 1

In [24]:
# Frequency
rfm["FrequencyScore"] = pd.qcut(rfm["Frequency"], 5, labels=[1,2,3,4,5])
# We divide the recency into 5 equal pieces and label them from min to max as 1 to 5 this time

In [25]:
# Monetary
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"], 5, labels=[1,2,3,4,5])

In [26]:
# RFM_SCORE
rfm["RFM_SCORE"] = (rfm["RecencyScore"].astype(str)+
                    rfm["FrequencyScore"].astype(str)+
                    rfm["MonetaryScore"].astype(str))

In [27]:
# To select the sleeping customers for example:
rfm[rfm["RFM_SCORE"]=="111"]

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12821.00,215,6,92.72,1,1,1,111
12831.00,263,9,215.05,1,1,1,111
12855.00,373,3,38.10,1,1,1,111
12897.00,205,4,216.50,1,1,1,111
12902.00,265,10,138.68,1,1,1,111
...,...,...,...,...,...,...,...
18224.00,264,10,158.95,1,1,1,111
18227.00,218,5,93.75,1,1,1,111
18269.00,367,7,168.60,1,1,1,111


NAMING & ANALYSING RFM SEGMENTS

In [28]:
# Naming RFM Segments

seg_map={
    r"[1-2][1-2]": "Hibernating",
    r'[1-2][3-4]': "At_Risk",
    r'[1-2][5]': "Can't_Loose",
    r"3[1-2]": "About_to_Sleep",
    r"33": "Need_Attention",
    r"[3-4][4-5]": "Loyal_Customers",
    r"41": "Promising",
    r"51": "New_Customers",
    r"[4-5][2-3]": 'Potential_Loyalists',
    r'5[4-5]': "Champions"
}
# Recency [1-2] - 1 or 2 and Frequency [1-2] - 1 or 2 == Hibernating

In [29]:
rfm["Segment"] = rfm["RecencyScore"].astype(str) + rfm["FrequencyScore"].astype(str)

In [30]:
# We need to assign the seg_map to the new segment variable now

rfm["Segment"] = rfm["Segment"].replace(seg_map, regex=True)

In [31]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])
# Now we have: Segments, number of customer in those segments and the metrics of those segments

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
About_to_Sleep        53.19   298     15.75   298   444.53   298
At_Risk              168.04   500     57.53   500   907.29   500
Can't_Loose          145.01    74    184.45    74  2258.58    74
Champions              6.45   555    282.99   555  6196.96   555
Hibernating          208.29   981     13.42   981   541.36   981
Loyal_Customers       34.47   722    155.65   722  2465.22   722
Need_Attention        52.47   188     41.82   188   858.87   188
New_Customers          7.22    49      7.04    49  4037.56    49
Potential_Loyalists   16.82   457     34.45   457   879.81   457
Promising             23.38    96      7.18    96   453.02    96

In [32]:
df[["Customer ID"]].nunique()
# we have 3921 customers - total

Customer ID    3921
dtype: int64

In [33]:
# We have 188 customers out of 3921 total customer who needs attention for example.
rfm[rfm["Segment"]=="Need_Attention"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12822.00,71,46,948.88,3,3,4,334,Need_Attention
12830.00,38,38,6814.64,3,3,5,335,Need_Attention
12842.00,71,34,1118.99,3,3,4,334,Need_Attention
12939.00,65,47,11581.80,3,3,5,335,Need_Attention
12944.00,36,30,604.51,3,3,3,333,Need_Attention


In [34]:
# We can now report those customers to the related departments

In [35]:
rfm[rfm["Segment"]=="Need_Attention"].index
# we can get the customer ids by doing indexing

Float64Index([12822.0, 12830.0, 12842.0, 12939.0, 12944.0, 13023.0, 13034.0,
              13107.0, 13109.0, 13165.0,
              ...
              18088.0, 18104.0, 18126.0, 18136.0, 18160.0, 18170.0, 18171.0,
              18192.0, 18205.0, 18228.0],
             dtype='float64', name='Customer ID', length=188)

In [36]:
# Reporting results to an excel file
new_df = pd.DataFrame()

In [37]:
new_df["Loyal_Customers"] = rfm[rfm["Segment"]=="Loyal_Customers"].index

In [38]:
new_df.to_excel("Loyal_Customers.xlsx")

In [39]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00,326,1,77183.60,1,1,5,115,Hibernating
12747.00,3,103,4196.01,5,4,5,545,Champions
12748.00,1,4596,33719.73,5,5,5,555,Champions
12749.00,4,199,4090.88,5,5,5,555,Champions
12820.00,4,59,942.34,5,4,4,544,Champions


In [40]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
About_to_Sleep        53.19   298     15.75   298   444.53   298
At_Risk              168.04   500     57.53   500   907.29   500
Can't_Loose          145.01    74    184.45    74  2258.58    74
Champions              6.45   555    282.99   555  6196.96   555
Hibernating          208.29   981     13.42   981   541.36   981
Loyal_Customers       34.47   722    155.65   722  2465.22   722
Need_Attention        52.47   188     41.82   188   858.87   188
New_Customers          7.22    49      7.04    49  4037.56    49
Potential_Loyalists   16.82   457     34.45   457   879.81   457
Promising             23.38    96      7.18    96   453.02    96

Now that we have summarised the snapshot of shopping behaviour of the customers for UK, the related departments can have business plans regarding marketing and sales actions.